In [2]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [3]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])

print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  15945
Number of unique ingredients:  7325
Number of total ingredients:  545617


### Ings
Ings df is just a database of unique ingredients and their harmful score, function and notes.

In [4]:
def dataPrep(ings, prods, prod_ing):
    #clean weird ingredient
    ings['ingredient'] = ings['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')
    #drop duplicates
    ings = ings.drop_duplicates(subset = 'ingredient')
    
    
    print('Number of unique ingredients: ', len(ings))

In [5]:
dataPrep(ings, prods, prod_ing)

Number of unique ingredients:  7324


In [3]:
ings.head()

Number of unique ingredients:  7324


,id,ingredient,ewg,cir,func_Abrasive,func_Antimicrobial,func_Antioxidant,func_Antistatic Agent,func_Astringent,func_Binding,...,notes_Comedogenic Rating (3),notes_Comedogenic Rating (4),notes_Comedogenic Rating (5),notes_Good for Dry Skin,notes_Good for Oily Skin,notes_Good for Sensitive Skin,notes_Paraben,notes_Promotes Wound Healing,notes_Sulfate,notes_UV Protection
0,0,Water,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Hydrogenated Polyisobutene,1,A,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Butylene Glycol,1,A,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,Sesamum Indicum (Sesame) Seed Oil,1,A,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
ing_uniqueID = ings.loc[:,['ingredient']].reset_index()
ing_uniqueID = ing_uniqueID.drop(['index'], axis = 1)
ing_uniqueID['uniqueID'] = ing_uniqueID.index
ing_uniqueID

### Prods
Prods df is a database of all the products with various information including brand, price and rating

In [5]:
#prods = prods.set_index('id')
#prods.loc[3477]
prods.head(3)

,id,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,cat_Cleansers,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
0,0,Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
2,2,Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,0,0,...,1,0,1,0,1,0,0,1,1,0
3,3,Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,0,0,...,0,0,1,1,0,0,1,1,1,0


In [6]:
product = 'Ultra Correction Line Repair Intensive Anti-Wrinkle Concentrate'
indices = pd.Series(prods.index, index=prods['product'])
idx = indices[product]
prods.loc[idx]['product']

'Ultra Correction Line Repair Intensive Anti-Wrinkle Concentrate'

In [7]:
idx

3477

In [8]:
prods.loc[3477]['product']

'Ultra Correction Line Repair Intensive Anti-Wrinkle Concentrate'

### Prod_ing
Prod ing is a database that essentiall links prods and ings. it contains the product id, ingredient and ingredient order.

In [9]:
prod_ing.loc[prod_ing['id'] == idx]

,id,ingredient,order
96616,3477,Water,1
96617,3477,Glycerin,2
96618,3477,Butylene Glycol,3
96619,3477,PPG-2 Myristyl Ether Propionate,4
96620,3477,Alcohol,5
96621,3477,Isohexadecane,6
96622,3477,Alumina,7
96623,3477,Methyl Methacrylate Crosspolymer,8
96624,3477,Silica,9
96625,3477,Ammonium Alum,10


In [10]:
#prod_ing = prod_ing.set_index('id')
prod_ing.head(3)

,id,ingredient,order
0,0,Water,1
1,0,Hydrogenated Polyisobutene,2
2,0,Butylene Glycol,3


### Add a column that is a unique ingredient identifier

In [11]:
#id is product id
#uniqueID is ingredient ID
prod_ing = pd.merge(prod_ing, ing_uniqueID, on='ingredient')
prod_ing = prod_ing.sort_values(['id','order'])
prod_ing.head()

,id,ingredient,order,uniqueID
0,0,Water,1,0
15954,0,Hydrogenated Polyisobutene,2,1
16617,0,Butylene Glycol,3,2
24563,0,Sesamum Indicum (Sesame) Seed Oil,4,3
24889,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,5,4


In [12]:
prod_ing.loc[prod_ing['id'] == idx]

,id,ingredient,order,uniqueID
2845,3477,Water,1,0
59148,3477,Glycerin,2,23
18214,3477,Butylene Glycol,3,2
520255,3477,PPG-2 Myristyl Ether Propionate,4,3703
125767,3477,Alcohol,5,63
284724,3477,Isohexadecane,6,328
401898,3477,Alumina,7,958
348619,3477,Methyl Methacrylate Crosspolymer,8,592
160346,3477,Silica,9,102
380167,3477,Ammonium Alum,10,793


## Reformatting Prod_ing
Instead of having each ingredient in it's own row, we want to create a list of ingredients for each product while retaining the order.

In [13]:
prod_ing.groupby('id')['ingredient'].apply(list)

id
0        [Water, Hydrogenated Polyisobutene, Butylene G...
1        [Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...
2        [Water, Glycerin, Cetearyl Alcohol, Petrolatum...
3        [Water, Glycereth-26, Alcohol, Butylene Glycol...
4        [Petrolatum, Glycerin, Cocamidopropyl Betaine,...
5        [Water, Butylene Glycol, Glycerin, Disodium Su...
6        [Mineral Oil, Avena Sativa (Oat) Kernel Flour,...
7        [Sodium Palm Kernelate, Water, Sodium Borate, ...
8        [Water, Hamamelis Virginiana (Witch Hazel) Wat...
9        [Water, Caprylic/Capric Triglyceride, Glycerin...
10       [Water, Glycerin, Sodium Laureth Sulfate, PEG-...
11       [Water, Tri-C14-15 Alkyl Citrate, Cyclohexasil...
12       [Water, Butylene Glycol, Propylene Glycol, Dip...
13       [Sodium Palm Kernelate, Water, Petrolatum, Gly...
14       [Water, Chamaecyparis Obtusa Water, Glycerin, ...
15       [Snail Secretion Filtrate, Water, Butylene Gly...
16       [Water, Bentonite, Glycerin, Magnesium Alumi

In [14]:
#cleanup of a rogue ingredient here as well
prod_ing['ingredient'] = prod_ing['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')

##group and create list
prod_ing_lists = prod_ing.groupby('id')['ingredient'].apply(list)
prod_ing_ID_lists = prod_ing.groupby('id')['uniqueID'].apply(list)

##convert back to dataframe and reset index
prod_ing_df = prod_ing_lists.to_frame().reset_index()
prod_ing_ID_df = prod_ing_ID_lists.to_frame().reset_index()
##check to make sure unique id's are still in tact (id shouldn't be exactly == to index)
prod_ing_lists.head()

id
0    [Water, Hydrogenated Polyisobutene, Butylene G...
1    [Mineral Oil, Glycerin, PEG-12, Cyclopentasilo...
2    [Water, Glycerin, Cetearyl Alcohol, Petrolatum...
3    [Water, Glycereth-26, Alcohol, Butylene Glycol...
4    [Petrolatum, Glycerin, Cocamidopropyl Betaine,...
Name: ingredient, dtype: object

In [15]:
prod_ing_df.loc[prod_ing_df['id'] == idx]

,id,ingredient
3466,3477,"[Water, Glycerin, Butylene Glycol, PPG-2 Myris..."


In [16]:
prod_ing_lists = pd.merge(prod_ing_df, prod_ing_ID_df, on='id')
prod_ing_lists = prod_ing_lists.rename(columns = {'ingredient': 'ingList', 'uniqueID': 'ing#List' })
prod_ing_lists.loc[prod_ing_lists['id'] == idx]

,id,ingList,ing#List
3466,3477,"[Water, Glycerin, Butylene Glycol, PPG-2 Myris...","[0, 23, 2, 3703, 63, 328, 958, 592, 102, 793, ..."


## Merge prod_ings and prods
We can now use the list to merge back with the products.

In [17]:
prod_ing_lists.loc[prod_ing_lists['id'] == idx]

,id,ingList,ing#List
3466,3477,"[Water, Glycerin, Butylene Glycol, PPG-2 Myris...","[0, 23, 2, 3703, 63, 328, 958, 592, 102, 793, ..."


In [18]:
prods.loc[prods['id'] == idx]

,id,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,cat_Cleansers,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
3477,3477,Ultra Correction Line Repair Intensive Anti-Wr...,Chanel,NaN,0.0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,1,1


In [19]:
products_and_ingredients= pd.merge(prod_ing_lists, prods, on = 'id')
#products_and_ingredients.tail(3)
#len(products_and_ingredients)
products_and_ingredients.loc[products_and_ingredients['id'] == idx]

,id,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
3066,3477,"[Water, Glycerin, Butylene Glycol, PPG-2 Myris...","[0, 23, 2, 3703, 63, 328, 958, 592, 102, 793, ...",Ultra Correction Line Repair Intensive Anti-Wr...,Chanel,NaN,0.0,0,0,1,...,0,1,0,1,1,0,0,0,1,1


### Add column for number of ingredients

In [20]:
products_and_ingredients['ingCount'] = products_and_ingredients['ingList'].apply(lambda x: len(x))
products_and_ingredients.loc[products_and_ingredients['id'] == idx]

,id,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,...,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount
3066,3477,"[Water, Glycerin, Butylene Glycol, PPG-2 Myris...","[0, 23, 2, 3703, 63, 328, 958, 592, 102, 793, ...",Ultra Correction Line Repair Intensive Anti-Wr...,Chanel,NaN,0.0,0,0,1,...,1,0,1,1,0,0,0,1,1,39


In [21]:
products_and_ingredients.to_csv('recommendations/recommender_products.csv')

# GET RECOMMENDATIONS


In [22]:
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]

df = products_and_ingredients
#remove water
df['ing#List'] = [remove_values_from_list(i,0) for i in df['ing#List']]

In [23]:
df.loc[df['id'] == idx]
df['id2'] = df['id']
df = df.reset_index()
df = df.drop(['id'], axis = 1)
df = df.rename(columns={'index': 'id'})
df.head(3)

,id,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,...,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount,id2
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,...,0,0,1,0,0,0,1,0,22,0
1,1,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...","[23, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 5...",Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,...,1,0,1,0,0,1,1,0,20,2
2,2,"[Water, Glycereth-26, Alcohol, Butylene Glycol...","[62, 63, 2, 64, 65, 66, 67, 68, 69, 70, 71, 72...",Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,...,1,1,0,0,1,1,1,0,38,3


In [24]:
product ='Facial Treatment Essence'
ingTest = df[df['product'].str.match(product)]
ingTest = ingTest['ingList']
print(*ingTest, sep='\n')

['Saccharomyces Ferment Filtrate', 'Butylene Glycol', 'Pentylene Glycol', 'Water', 'Sodium Benzoate', 'Methylparaben', 'Sorbic Acid']


## Cosine similarity

Reference [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

In [32]:
%%time

recommended = get_COS_recommendations(df,product)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/COS.csv')

ValueError: additive_chi2 does not support sparse matrices.

In [33]:
topTen

,product,ingList,commonIng
86,Pure Maracuja Oil,[Passiflora Edulis Seed Oil],0
236,100% Plant-Derived Squalane,[Squalane],0
405,Squalane 100% Pure,[Squalane],0
829,Thermal Spring Water,"[Water, Nitrogen]",1
900,Secret Indian Healing Clay,[Bentonite],0
1025,Witch Hazel Astringent,[Hamamelis Virginiana (Witch Hazel],0
1062,100% Pure Tea Tree Oil,[Melaleuca Alternifolia (Tea Tree) Leaf Oil],0
1169,Organic Rose Hip Oil,[Rosa Canina Fruit Oil],0
1198,Olive Virgin Oil,[Olea Europaea (Olive) Fruit Oil],0
1426,Virgin Marula Luxury Facial Oil,[Sclerocarya Birrea (Marula) Oil],0


## Average Overlap

In [30]:
%%time

recommended = get_A0_recommendations(df,product)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/AO.csv')

ZeroDivisionError: division by zero

In [ ]:
topTen

## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

In [29]:
#%%time

recommended = get_RBO_recommendations(df, product, rbo_min)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/RBO.csv')

ZeroDivisionError: division by zero

In [28]:
topTen

,product,ingList,commonIng
86,Pure Maracuja Oil,[Passiflora Edulis Seed Oil],0
236,100% Plant-Derived Squalane,[Squalane],0
405,Squalane 100% Pure,[Squalane],0
829,Thermal Spring Water,"[Water, Nitrogen]",1
900,Secret Indian Healing Clay,[Bentonite],0
1025,Witch Hazel Astringent,[Hamamelis Virginiana (Witch Hazel],0
1062,100% Pure Tea Tree Oil,[Melaleuca Alternifolia (Tea Tree) Leaf Oil],0
1169,Organic Rose Hip Oil,[Rosa Canina Fruit Oil],0
1198,Olive Virgin Oil,[Olea Europaea (Olive) Fruit Oil],0
1426,Virgin Marula Luxury Facial Oil,[Sclerocarya Birrea (Marula) Oil],0
